<a href="https://colab.research.google.com/github/nargis78390-creator/Fake_Review_detection_project/blob/main/MinorProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

importing libraries

In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import string
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

to connect google drive and google colab

In [4]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

dataset is loaded from a CSV file

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/fake reviews dataset.csv')

In [ ]:
length=len(df)
print(length)



```
# This is formatted as code
```

Check the first 100 rows of the dataset

In [ ]:
df.head(100)

check the last five rows of the dataset

In [ ]:
df.tail(20)

Check for any missing values


In [ ]:
df.isnull().sum()

assess the structure and quality of your DataFrame

In [ ]:
df.info()

 for a quick statistical summary of numerical data

In [ ]:
df.describe()

In [ ]:
df['rating'].value_counts()

removing punctuation and stopwords

In [ ]:
def clean_text(text):
    nopunc = [w for w in text if w not in string.punctuation]
    nopunc = ''.join(nopunc)
    return  ' '.join([word for word in nopunc.split() if word.lower() not in stopwords.words('english')])

In [ ]:
df['text_'][0], clean_text(df['text_'][0])

In [ ]:
df['text_'].head().apply(clean_text)

In [ ]:
df.shape

In [ ]:
df['text_'] = df['text_'].astype(str)

In [ ]:
def preprocess(text):
    return ' '.join([word for word in word_tokenize(text) if word not in stopwords.words('english') and not word.isdigit() and word not in string.punctuation])

In [ ]:
import nltk
nltk.download('punkt_tab')


In [ ]:
preprocess(df['text_'][4])

In [ ]:
df['text_'][:10000] = df['text_'][:10000].apply(preprocess)

In [ ]:
df['text_'][10001:20000] = df['text_'][10001:20000].apply(preprocess)

In [ ]:
df['text_'][20001:30000] = df['text_'][20001:30000].apply(preprocess)

In [ ]:
df['text_'][30001:40000] = df['text_'][30001:40000].apply(preprocess)

In [ ]:
df['text_'][40001:40432] = df['text_'][40001:40432].apply(preprocess)

In [ ]:
df['text_'] = df['text_'].str.lower()

In [ ]:
stemmer = PorterStemmer()
def stem_words(text):
    return ' '.join([stemmer.stem(word) for word in text.split()])
df['text_'] = df['text_'].apply(lambda x: stem_words(x))

In [ ]:
lemmatizer = WordNetLemmatizer()
def lemmatize_words(text):
    return ' '.join([lemmatizer.lemmatize(word) for word in text.split()])
df["text_"] = df["text_"].apply(lambda text: lemmatize_words(text))


In [ ]:
df['text_'].head()


In [ ]:
df.to_csv('Preprocessed Fake Reviews Detection Dataset.csv')



In [ ]:
df.columns = df.columns.str.strip()


In [ ]:
df.head()

Apply cleaning function to your text column

In [ ]:
import re
import string

# Define the clean_text function to process the text
def clean_text(text):
    # Remove punctuation
    text = re.sub(f"[{string.punctuation}]", "", text)
    # Convert text to lowercase
    text = text.lower()
    # Remove digits
    text = re.sub(r'\d+', '', text)
    # Remove extra whitespace
    text = text.strip()
    return text


In [ ]:
df['cleaned_reviews'] = df['text_'].apply(clean_text)


In [ ]:
df.drop(columns=['category','rating','text_'],inplace=True,errors='ignore')


In [ ]:
df

In [ ]:
# draw wordcloud (for fake review)

from wordcloud import WordCloud
import matplotlib.pyplot as plt
text = ' '.join(word for word in df['cleaned_reviews'][df['label'] == 'CG'].astype(str))
wordcloud = WordCloud(width=1000, height=600,max_font_size = 100)
plt.figure(figsize=(15,12))
plt.imshow(wordcloud.generate(text), interpolation='bilinear')
plt.axis('off')
plt.show()

In [ ]:
# draw wordcloud (for original or real review)

text = ' '.join(word for word in df['cleaned_reviews'][df['label'] == 'OR'].astype(str))
wordcloud = WordCloud(width=1000, height=600,max_font_size = 100)
plt.figure(figsize=(15,12))
plt.imshow(wordcloud.generate(text), interpolation='bilinear')
plt.axis('off')
plt.show()

In [ ]:
#input
x= df['cleaned_reviews']
#output
y = df['label']

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [ ]:
#TfidVectorizer --> TF(Term Freq) * IDF (Inverse Doc Freq)

In [ ]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
x_train

In [ ]:
y_train

Embedding with Sentence Transformers

In [ ]:
# Install sentence-transformers library
!pip install -q sentence-transformers

# Import model
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Load pre_trained model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Make sure all reviews are string type
df['cleaned_reviews'] = df['cleaned_reviews'].astype(str)

# Convert text to dense embeddings
embeddings = model.encode(df['cleaned_reviews'].tolist(), show_progress_bar=True)

#label encoding (Prepare target)
label_map = {'CG': 0, 'OR': 1}
df['label'] = df['label'].map(label_map)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(embeddings, df['label'], test_size=0.2, random_state=42)

# Train Logistic Regression  model
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)

# Evaluate model performance
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred, target_names=['CG', 'OR']))


In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# Generate confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Display the confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['CG (Fake)', 'OR (Real)'])
disp.plot(cmap=plt.cm.Blues)
plt.title("Confusion Matrix for Fake Review Detection")
plt.show()


In [ ]:
plt.savefig("confusion_matrix.png", dpi=300)


In [ ]:
from google.colab import files
files.download("confusion_matrix.png")


In [ ]:
# Plot ROC curve after confusion matrix

from sklearn.metrics import roc_curve, roc_auc_score
import matplotlib.pyplot as plt

# Get scores (not hard predictions) for ROC
y_pred_proba = clf.predict_proba(X_test)[:, 1]

# Calculate ROC curve
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)

# Calculate AUC
auc = roc_auc_score(y_test, y_pred_proba)

# Plot ROC curve
plt.figure()
plt.plot(fpr, tpr, label='Logistic Regression (AUC = {:.2f})'.format(auc))
plt.plot([0, 1], [0, 1], 'k--')  # Diagonal for baseline
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc='lower right')
plt.grid()
plt.savefig("roc_curve.png", dpi=300)
plt.show()


In [ ]:
plt.savefig("roc_curve.png", dpi=300)


In [ ]:
from google.colab import files
files.download("roc_curve.png")


In [ ]:
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import pandas as pd

# Get classification report as a dictionary
report = classification_report(y_test, y_pred, target_names=['CG (Fake)', 'OR (Real)'], output_dict=True)

# Convert to DataFrame for plotting
df_report = pd.DataFrame(report).transpose()

# Only keep classes, not 'accuracy', 'macro avg', etc.
df_report = df_report.iloc[:2][['precision', 'recall', 'f1-score']]

# Plot
df_report.plot(kind='bar', figsize=(8, 6))
plt.title('Precision, Recall, and F1-Score per Class')
plt.ylabel('Score')
plt.ylim(0, 1.0)
plt.xticks(rotation=0)
plt.grid(axis='y')
plt.tight_layout()
plt.savefig("f1_scores_barplot.png", dpi=300)
plt.show()


In [ ]:
from google.colab import files
files.download("f1_scores_barplot.png")


In [ ]:
#install a python library for creating simple web apps
!pip install -q --upgrade gradio

#import gradio for building interface
import gradio as gr

#define function
def review_checker(Review):
    emb = model.encode([Review])# Convert input review to sentence embedding
    pred = clf.predict(emb)[0]  # Predict label using trained model
    return "Fake" if pred == 1 else "Real" # return output

gr.Interface(fn=review_checker, inputs="textbox", outputs="label", title="Fake Review Detector").launch(share=True)


In [ ]:
!pip install nbstripout
!nbstripout MinorProject.ipynb
